In [103]:
import csv
import json
import time
import requests
import numpy as np
import pandas as pd
from random import randint

from pyproj import Transformer
from geopy.distance import geodesic
from geopy.geocoders import Nominatim

### 1. Get Addresses from geoportal.staedteregion

In [3]:
url = "https://geoportal.staedteregion-aachen.de/GnUrlProxy?url=32026040&btl=/json/search/strhsnr/strasse/2"
contents = requests.get(url)
aachen_streets = contents.json()
aachen_street_json_path = "aachen_street.json"
json.dump(aachen_streets, open(aachen_street_json_path, 'w')) # dont call it multiple times, so temporarily save data to json file

### 2. Grab Data from https://geoportal.staedteregion-aachen.de/GnUrlProxy?url=32026040&btl=/json/search/strhsnr/hausnummer/2/{stree_id}

In [5]:
# load the previously saved json
file = open(aachen_street_json_path, 'r', encoding='utf-8')
data = json.load(file)

csv_save_path = "aachen_house_no.csv"
csvfile = open(csv_save_path, 'w', newline='', encoding='utf-8')
writer = csv.writer(csvfile)
writer.writerow(["id", "street", "house_no", "geom"]) 

house_no_url_prefix = "https://geoportal.staedteregion-aachen.de/GnUrlProxy?url=32026040&btl=/json/search/strhsnr/hausnummer/2/"
for street in aachen_streets[:100]:
    time.sleep(1) # rest 1 seconds for each request
    house_no_contents = requests.get(house_no_url_prefix+street["id"])
    
    try:
      house_nos = house_no_contents.json()

    except json.JSONDecodeError:
      print(f'street id: {street["id"]} has no content')
      continue

    for item in house_nos:
      writer.writerow([street["id"], street["bez"], item["nr"], item["geom"]])

street id: =2550 has no content
street id: =2555 has no content
street id: =2564 has no content
street id: =2563 has no content
street id: =2552 has no content
street id: =2553 has no content


### 3. Get Longitude, Latitude, Zipcode

In [ ]:
geolocator = Nominatim(user_agent="my_project")

def get_zipcode(lat, lon):
    location = geolocator.reverse((lat, lon), exactly_one=True)
    time.sleep(randint(1 ,3))
    zip_code = 0
    # Extract zip code
    if location and "postcode" in location.raw["address"]:
        zip_code = location.raw["address"]["postcode"]
    print("Zip code:", zip_code)
    return zip_code

df = pd.read_csv("aachen_house_no.csv")

# house_no "Anzeigen" or "0" are invalid
excluded_values = ["Anzeigen", "0"]
df = df[~df['house_no'].isin(excluded_values)]

df[['x', 'y']] = df['geom'].str.split(expand=True).astype(float)
transformer = Transformer.from_crs("EPSG:25832", "EPSG:4326", always_xy=True)
df['longitude'], df['latitude'] = zip(*df.apply(lambda row: transformer.transform(row['x'], row['y']), axis=1))
df = df.drop(columns=['x', 'y'])

for i in df.index:
    lat = df.at[i, 'latitude']
    lon = df.at[i, 'longitude']
    zip_code = get_zipcode(lat, lon)
    df.at[i, 'zip_code'] = zip_code

    if i % 100 == 0:
        df.to_csv('progress_aachen_house_no.csv', index=False)
        print(f"Progress saved at row {i}")

Zip code: 52076
Progress saved at row 0
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076
Zip code: 52076


In [40]:
df.to_csv('aachen_house_no_zip_code.csv', index=False)

In [43]:
df

,id,street,house_no,geom,longitude,latitude,zip_code
0,3200.0,Aachener Straße,11,298136.611 5624134.902,6.139304,50.733879,52076
1,3200.0,Aachener Straße,12,298111.01 5624098.004,6.138962,50.733538,52076
2,3200.0,Aachener Straße,13,298145.099 5624146.249,6.139418,50.733984,52076
3,3200.0,Aachener Straße,60,298224.179 5623600.616,6.140835,50.729110,52076
4,3200.0,Aachener Straße,69,298262.568 5623535.223,6.141414,50.728536,52076
...,...,...,...,...,...,...,...
3288,3599.0,An der Unterbahn,44,299466.054 5626393.473,6.156890,50.754625,52078
3289,3599.0,An der Unterbahn,45,299418.865 5626419.721,6.156207,50.754845,52078
3290,3599.0,An der Unterbahn,46,299470.892 5626397.662,6.156956,50.754665,52078
3291,3599.0,An der Unterbahn,47,299412.731 5626426.023,6.156117,50.754899,52078


### 4. Only Select One House No. per Zipcode

In [77]:
df = pd.read_csv("aachen_house_no_zip_code.csv")

df = df.copy()
df = df.drop_duplicates(subset=["zip_code"], keep='first')
df["zip_code"] = df["zip_code"].astype(int)

In [78]:
df = df.reset_index(drop=True)

### 5. Project Longitude, Latitude to X-Y Coordinates Using WGS 84 to Web Mercator

In [98]:
projector = Transformer.from_crs("EPSG:4326", "EPSG:3857")  # WGS 84 to Web Mercator
def lon_lat_to_xy(longitude, latitude):
    return projector.transform(longitude, latitude)

final_df = df.copy()
final_df[['x', 'y']] = df.apply(lambda col: lon_lat_to_xy(col['longitude'], col["latitude"]), axis=1, result_type='expand')
final_df

,id,street,house_no,geom,longitude,latitude,zip_code,x,y
0,3200.0,Aachener Straße,11,298136.611 5624134.902,6.139304,50.733879,52076,5.647670e+06,684735.702468
1,2230.0,Aachener-und-Münchener-Allee,1,293683.424 5627354.953,6.074479,50.761234,52074,5.650715e+06,677478.268930
2,3632.0,AachenMünchener-Platz,1,294621.785 5628398.439,6.087184,50.770939,52064,5.651795e+06,678900.586131
3,3663.0,Abteiblick,2,294745.197 5627604.753,6.089375,50.763854,52066,5.651006e+06,679145.826984
4,1004.0,Achterstraße,2,294692.725 5629589.182,6.087524,50.781658,52062,5.652988e+06,678938.654597
5,2000.0,Adalbertsteinweg,1,295306.477 5628794.884,6.096661,50.774741,52070,5.652218e+06,679961.535810
6,1005.0,Adalbert-Stifter-Straße,1,298620.846 5626158.014,6.145051,50.752218,52078,5.649711e+06,685379.213666
7,2901.0,Adele-Weidtman-Straße,1,293287.105 5631589.842,6.066487,50.799127,52072,5.654933e+06,676583.527989
8,2801.0,Ahornweg,2,297604.276 5631214.509,6.127876,50.797280,52080,5.654727e+06,683456.285653
9,1019.0,Alsenstraße,1,296552.248 5628431.893,6.114507,50.771919,52068,5.651904e+06,681959.541325


### 6. Calculate Distance Using geopy.geocoders Lib

In [99]:
def calculate_distance_from_each_base(coord1, lat, lon):
    coord2 = (lat, lon)   # latitude and longitude of point 2
    return geodesic(coord1, coord2).km

# depot_row  = final_df[final_df['zip_code'] == 52076] # assume the address with this zip code is our depot
# DEPOT_COORDS = (depot_row['latitude'].values[0], depot_row['longitude'].values[0])

# final_df['distance_km_from_depot'] = final_df.apply(lambda x: calculate_distance_from_each_base(DEPOT_COORDS, x['latitude'], x['longitude']), axis=1)
for i, _ in final_df.iterrows():
  base_coords = (final_df['latitude'][i], final_df['longitude'][i],)
  final_df[f'distance_km_from_{i}'] = final_df.apply(
            lambda x: calculate_distance_from_each_base(base_coords, x['latitude'], x['longitude']),
            axis=1
        )

### 7. Demand Generation (600-20,000)

In [108]:
final_df['demand_kg'] = np.random.randint(600, 20000, size=len(df))
final_df.loc[final_df['zip_code'] == 52076, 'demand_kg'] = 0 # depot

In [109]:
final_df

,id,street,house_no,geom,longitude,latitude,zip_code,x,y,distance_km_from_0,distance_km_from_1,distance_km_from_2,distance_km_from_3,distance_km_from_4,distance_km_from_5,distance_km_from_6,distance_km_from_7,distance_km_from_8,distance_km_from_9,demand_kg
0,3200.0,Aachener Straße,11,298136.611 5624134.902,6.139304,50.733879,52076,5.647670e+06,684735.702468,0.000000,5.494804,5.524951,4.851436,6.449841,5.451486,2.080050,8.892471,7.098870,4.579297,0
1,2230.0,Aachener-und-Münchener-Allee,1,293683.424 5627354.953,6.074479,50.761234,52074,5.650715e+06,677478.268930,5.494804,0.000000,1.403179,1.090631,2.451331,2.169467,5.079873,4.252867,5.501129,3.063949,6494
2,3632.0,AachenMünchener-Platz,1,294621.785 5628398.439,6.087184,50.770939,52064,5.651795e+06,678900.586131,5.524951,1.403179,0.000000,0.803129,1.192714,0.791092,4.583389,3.458832,4.101437,1.930534,15202
3,3663.0,Abteiblick,2,294745.197 5627604.753,6.089375,50.763854,52066,5.651006e+06,679145.826984,4.851436,1.090631,0.803129,0.000000,1.984889,1.315692,4.136427,4.242948,4.604346,1.987134,14797
4,1004.0,Achterstraße,2,294692.725 5629589.182,6.087524,50.781658,52062,5.652988e+06,678938.654597,6.449841,2.451331,1.192714,1.984889,0.000000,1.003677,5.215092,2.444782,3.334121,2.189991,19132
5,2000.0,Adalbertsteinweg,1,295306.477 5628794.884,6.096661,50.774741,52070,5.652218e+06,679961.535810,5.451486,2.169467,0.791092,1.315692,1.003677,0.000000,4.234892,3.447725,3.336472,1.297433,10585
6,1005.0,Adalbert-Stifter-Straße,1,298620.846 5626158.014,6.145051,50.752218,52078,5.649711e+06,685379.213666,2.080050,5.079873,4.583389,4.136427,5.215092,4.234892,0.000000,7.611875,5.157151,3.073707,9336
7,2901.0,Adele-Weidtman-Straße,1,293287.105 5631589.842,6.066487,50.799127,52072,5.654933e+06,676583.527989,8.892471,4.252867,3.458832,4.242948,2.444782,3.447725,7.611875,0.000000,4.332963,4.541915,16540
8,2801.0,Ahornweg,2,297604.276 5631214.509,6.127876,50.797280,52080,5.654727e+06,683456.285653,7.098870,5.501129,4.101437,4.604346,3.334121,3.336472,5.157151,4.332963,0.000000,2.974533,18127
9,1019.0,Alsenstraße,1,296552.248 5628431.893,6.114507,50.771919,52068,5.651904e+06,681959.541325,4.579297,3.063949,1.930534,1.987134,2.189991,1.297433,3.073707,4.541915,2.974533,0.000000,954


In [110]:
final_df.to_csv('aachen_address_with_distance.csv')